In [69]:


import pandas as pd
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import LabelEncoder

In [70]:
#get the training dataset
raw_data = pd.read_csv('train.csv')

In [71]:
#drop few columns
data = raw_data.drop(['Name','Ticket','Cabin','PassengerId'],axis=1)

In [72]:
#removing na values

data = data.dropna()


In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 8 columns):
Survived    712 non-null int64
Pclass      712 non-null int64
Sex         712 non-null object
Age         712 non-null float64
SibSp       712 non-null int64
Parch       712 non-null int64
Fare        712 non-null float64
Embarked    712 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 50.1+ KB


In [61]:
#set missing age

#data["Age"].fillna(data.groupby("Pclass")["Age"].transform("mean"), inplace=True)

In [62]:
def setIsChild(age):
    if age < 10:
        return 1
    else:
        return 0
#data.head()

In [63]:
IsChild = data.Age.apply(setIsChild)
data['IsChild'] = IsChild


In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 9 columns):
Survived    712 non-null int64
Pclass      712 non-null int64
Sex         712 non-null object
Age         712 non-null float64
SibSp       712 non-null int64
Parch       712 non-null int64
Fare        712 non-null float64
Embarked    712 non-null object
IsChild     712 non-null int64
dtypes: float64(2), int64(5), object(2)
memory usage: 55.6+ KB


In [65]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data.Sex = le.fit_transform(data.Sex)
data.Embarked = le.fit_transform(data.Embarked)

In [66]:
X=data.drop('Survived',axis=1)
y = data['Survived']

In [67]:
#train all the models
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()

In [68]:
#ensemble with voting = hard
eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
eclf1 = eclf1.fit(X, y)


In [53]:
# getting test data

X_test = pd.read_csv('test.csv')
passenger_id = X_test.PassengerId
X_test = X_test.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)

# filling in missing age and fare by mean of age and fare in the pclass respectively
X_test["Age"].fillna(X_test.groupby("Pclass")["Age"].transform("mean"), inplace=True)
X_test["Fare"].fillna(X_test.groupby("Pclass")["Fare"].transform("mean"), inplace=True)

# compute Ischild column
age = X_test.Age.apply(setIsChild)
X_test['IsChild'] = age

#transform categorical columns
X_test.Sex = le.fit_transform(X_test.Sex)
X_test.Embarked = le.fit_transform(X_test.Embarked)

In [54]:
predictions = eclf1.predict(X_test)

In [55]:
output = pd.DataFrame({'PassengerId':passenger_id,'Survived':predictions})
#ensemble voting
output.to_csv('titanic_output_EnsembleVoting.csv',index=False)